In [97]:
#clean text and create stoplist, noun,proper,adj,verb remain
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models, similarities
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import csv
import re
def clean_text(text):
    text = text.replace("\n", " ") #newline
    text = re.sub(r"-", " ", text) #"-"
    text = re.sub(r"\d+/\d+/\d+", "", text) #timestamp
    text = re.sub(r"[\w]+@[\.\w]+", "", text) #ML
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text)
    pure_text = ""
    for letter in text:
        if letter.isalpha() or letter == " ":
            pure_text+=letter
    text = " ".join(word for word in pure_text.split() if len(word) > 1)
    return text

stoplist = pd.read_csv("/Users/CM/desktop/LDA2/stopqr.csv")["BodyText"].values

print("done.")

done.


In [98]:
#csv→clean_text→clean_stoppoint→into_corpora
# Gensim

df = pd.read_csv("/Users/CM/Desktop/LDA2/ldaqrtxt.csv")
#df = df[["Id","BodyText"]].dropna()
df = df[["BodyText"]].dropna()

docs = df["BodyText"]
docs = docs.apply(lambda s: clean_text(s))
doclist = docs.values

#print(type(doclist))

texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]

#print(texts)

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

print("done.")

done.


In [99]:
import logging
logging.basicConfig(filename='gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)
print("done.")

done.


In [131]:
#training

num_topics = 10
chunksize = 6000
passes = 100
iterations = 400
eval_every = None

lda = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,chunksize=chunksize,
                                      alpha='auto', eta='auto',eval_every=eval_every,
                                      iterations=iterations,passes = passes,num_topics = num_topics)

lda.print_topics(num_topics = num_topics, num_words = 25)

[(0,
  '0.008*"selection" + 0.006*"natural" + 0.005*"autism" + 0.005*"artificial" + 0.003*"brain" + 0.003*"autistic" + 0.003*"ecology" + 0.003*"development" + 0.003*"answer" + 0.003*"life" + 0.002*"question" + 0.002*"isnt" + 0.002*"read" + 0.002*"mit" + 0.002*"children" + 0.002*"aging" + 0.002*"disease" + 0.002*"book" + 0.002*"plants" + 0.002*"rate" + 0.002*"hard" + 0.002*"patent" + 0.002*"company" + 0.002*"changing" + 0.002*"idea"'),
 (1,
  '0.006*"babies" + 0.006*"off" + 0.005*"designer" + 0.004*"parents" + 0.003*"children" + 0.003*"baby" + 0.003*"effects" + 0.003*"child" + 0.003*"question" + 0.003*"embryos" + 0.003*"life" + 0.003*"genetically" + 0.003*"germline" + 0.003*"society" + 0.003*"start" + 0.003*"disease" + 0.003*"create" + 0.003*"perfect" + 0.002*"mental" + 0.002*"change" + 0.002*"answer" + 0.002*"future" + 0.002*"based" + 0.002*"ivf" + 0.002*"traits"'),
 (2,
  '0.010*"intelligence" + 0.009*"children" + 0.008*"parents" + 0.006*"life" + 0.005*"child" + 0.003*"genetics" + 0.0

In [19]:
########################GENSIM MODELS#############################
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        #model = LdaMallet(mallet_path, id2word=dictionary, corpus=corpus, num_topics=num_topics)
        #HdpModel(corpus=corpus, id2word=dictionary)
        #LdaMallet(mallet_path, id2word=dictionary, corpus=corpus,iterations=30 , num_topics=num_topics)
        model = Ldamodel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        #LsiModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        topics = [[word for word, prob in topic] for topicid, topic in model.show_topics(num_topics=num_topics, formatted=False)]
        coherencemodel = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v',window_size=200)
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

lmt=100; strt=10; stp=10

model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus, texts=texts, start=strt, limit=lmt, step=stp)

NameError: name 'Ldamodel' is not defined

In [146]:
import re
import matplotlib.pyplot as plt
p = re.compile("(-*\d+\.\d+) per-word .* (\d+\.\d+) perplexity")
matches = [p.findall(l) for l in open('gensim.log')]
matches = [m for m in matches if len(m) > 0]
tuples = [t[0] for t in matches]
perplexity = [float(t[1]) for t in tuples]
liklihood = [float(t[0]) for t in tuples]
iter = list(range(0,len(tuples)*10,10))
plt.plot(iter,liklihood,c="black")
plt.ylabel("log liklihood")
plt.xlabel("iteration")
plt.title("Topic Model Convergence")
plt.grid()
plt.savefig("convergence_liklihood.pdf")
plt.close()

#print('\nPerplexity: ', lda.log_perplexity(corpus))

In [137]:
#print('\nPerplexity: ', lda.log_perplexity(corpus))

In [ ]:
#coherence_model_lda = CoherenceModel(model=lda, texts=corpus, dictionary=dictionary, coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

In [1]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

NameError: name 'models' is not defined